In [3]:
import numpy as np

In [16]:
def getdata():
    data = np.genfromtxt('train12.txt')
    x = data[:,:-1]
    y = data[:,-1]
    return x, y

def testData():
    data = np.genfromtxt('test12.txt')
    x = data[:,:-1]
    y = data[:,-1]
    return x,y

def hfunc(x, s, theta):
    if s:
        return sign(x - theta)
    else:
        return -sign(x - theta)
    
def sign(x):
    if x>=0:
        return 1
    else:
        return -1
    
def updateweight(x,y,theta,dim,s,weights,h,epsilon):
    
    #根據公式更新Un
    for i in range(len(x)):
        if y[i] != h(x[i][dim], s, theta):
            weights[i] *= epsilon
        else:
            weights[i] /= epsilon
    return weights
    
def weightedErrorRate(x,y,s,h,theta,dim,weights):
    
    #根據公式計算Ein
    error = 0
    for i in range(len(x)):
        if y[i] != h(x[i][dim], s, theta):
            error += weights[i]
    return error / np.sum(weights)

def trainDecisionStump(x,y,weights):
    dimension = len(x[0]) # dimension = feature的個數
    E_in = 1
    best_s = True
    best_theta = 0
    best_dim = 0
    for dim in range(dimension):
        #對每一個dimension做decision stump
        thetas = np.sort(x[:,dim]) # 排序某一列 x 作爲 candidate theta
        ss = [True, False]
        for theta in thetas:
            for s in ss:
                E = weightedErrorRate(x,y,s,hfunc,theta,dim,weights)
                if E < E_in:
                    E_in = E
                    best_s = s
                    best_theta = theta
                    best_dim = dim
    return best_s, best_theta, best_dim, E_in

def adaboost(trainX, trainY, h, weights, T):
    alphas = []  
    g_funcs = []  # gt
    min_error = 10000
    for i in range(T):
        # 有T筆資料
        s, theta, dim, error = trainDecisionStump(trainX, trainY, weights) # 第 i (t) 個 hypothesis
        g_funcs.append([s, theta, dim]) # 將訓練第 t 筆資料得到的 hypothesis 保存，實際上等於 ppt 上的 gt
        epsilon = np.sqrt((1 - error) / error)
        alphas.append(np.log(epsilon))
        
        #if i %50 == 49:
                #print('iteration:',i+1)
                
        min_error = min(error,min_error)
        
        #print('iteration ',i+1,':','sum of weights:',np.sum(weights))
        #print('error:',error)
        
        weights = updateweight(trainX,trainY,theta,dim,s,weights,h,epsilon)
    
    print('minimum of error is:',min_error)
        
    return alphas, g_funcs

def gError(x,y,h,s,theta,dim):
    err = 0
    for i in range(len(y)):
        if y[i] != h(x[i][dim], s, theta):
            err +=1
    return err / len(y)

def Gx(alphas, x, gfuncs, h):
    G = 0
    for t in range(len(alphas)):
        s, theta, dim = gfuncs[t]
        G += alphas[t]*h(x[dim], s, theta)
    return sign(G)

def boostError(alphas, x, gfuncs, h, y):
    err = 0
    for i in range(len(x)):
        if Gx(alphas, x[i], gfuncs, h) != y[i]:
            err+=1        
    return err / len(y)


def main():
    trainX, trainY = getdata()
    testX, testY = testData()
    weights = np.array([1/len(trainY)]*len(trainY))
    T = 300
    alphas, g_funcs = adaboost(trainX, trainY, hfunc, weights, T)
    E_g1 = gError(trainX, trainY, hfunc, g_funcs[0][0], g_funcs[0][1], g_funcs[0][2])
    print("Error rate of g1 : ", E_g1)
    E_G = boostError(alphas, trainX, g_funcs, hfunc, trainY)
    print("Error rate of G :", E_G)
    E_out_g1 = gError(testX, testY, hfunc, g_funcs[0][0], g_funcs[0][1], g_funcs[0][2])
    print("Eout of g1 : ", E_out_g1)
    E_out_G = boostError(alphas, testX, g_funcs, hfunc, testY)
    print("Eout of G :", E_out_G)
    
    

In [68]:
# Ridge Regression for classification

def getdata1():
    dt = np.genfromtxt('train19.txt')
    x = dt[:,:-1]
    y = dt[:,-1]
    train_x = x[:400]
    train_y = y[:400]
    test_x = x[400:]
    test_y = y[400:]
    return train_x, train_y, test_x, test_y

def kernel(x1, x2, gamma):
    # x1,x2是 vector
    delta = x1 - x2
    k = np.sqrt(np.sum(delta*delta))
    return np.exp(-gamma*k)

def kernelMatrix(X1,X2,gamma):
    N = len(X1)
    M = len(X2)
    K_matrix = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            K_matrix[i][j] = kernel(X1[i], X2[j], gamma)
    return K_matrix

def solvebeta(y, K, lamda):
    N = len(K)
    identity = np.identity(N)
    beta = np.linalg.inv(lamda*identity + K)
    return beta.dot(y)

def sign(x):
    if x<0:
        return -1
    else:
        return 1

def errorRate(beta, K, y):
    res = K.T.dot(beta)
    res = list(map(sign,res))
    return np.sum(y!=res) / len(y)

def main19():
    train_x, train_y, test_x, test_y = getdata1()
    gamma_list = [32, 2, 0.125]
    lamda_list = [0.001, 1, 1000]
    best_Ein = 1
    Eout = []
    best_gamma = 1
    best_lamda = 1
    best_beta = None
    for gamma in gamma_list:
        K = kernelMatrix(train_x, train_x, gamma)
        K2 = kernelMatrix(train_x, test_x, gamma)
        for lamda in lamda_list:
            beta = solvebeta(train_y, K, lamda)
            Ein = errorRate(beta, K, train_y)
            if Ein < best_Ein:
                best_gamma = gamma
                best_lamda = lamda
                best_beta = beta
                best_Ein = Ein
            eout = errorRate(beta, K2, test_y)
            Eout.append(eout)
    print('best Ein : ', best_Ein)
    print('best gamma : ', best_gamma)
    print('best lamda :', best_lamda)
    print('best eout :', min(Eout))

In [69]:
main19()

best Ein :  0.0
best gamma :  32
best lamda : 0.001
best eout : 0.38
